# Score Calculator
Automatically computes the **Coherence Score Φ** from your indicators and latest data.

Prerequisites:
```bash
pip install pandas pyyaml
```

Folder layout assumed:
```
community-conflict-guide/
  indicators.yaml
  data/
      processed/
          *.csv  # one CSV per indicator, latest measurement already aggregated
```

In [ ]:
import pandas as pd
import yaml, json, glob, os
from pprint import pprint


In [ ]:
# Load indicator definitions
with open('indicators.yaml', 'r') as f:
    ind = yaml.safe_load(f)
print('Indicators:')
pprint(ind)


In [ ]:
data_dir = 'data/processed'
values = {}
for metric in ind:
    file_pattern = os.path.join(data_dir, f'{metric}*.csv')
    files = sorted(glob.glob(file_pattern))
    if not files:
        raise FileNotFoundError(f'No data file found for {metric}')
    # use the latest by alphabetical order (timestamp prefix recommended)
    latest = files[-1]
    df = pd.read_csv(latest)
    # expecting a single value column called 'value'
    val = df['value'].iloc[-1]
    values[metric] = float(val)

print('Latest measurements:')
pprint(values)


In [ ]:
mismatches = {}
for metric, info in ind.items():
    current = values[metric]
    target = info['target']
    weight = info['weight']
    mismatch = abs(current - target) / target
    mismatches[metric] = {'value': current, 'target': target, 'weight': weight, 'mismatch': mismatch}

score = sum(d['weight'] * d['mismatch'] for d in mismatches.values())
print(f'\nCoherence Score Φ = {score:.4f}\n')
pd.DataFrame(mismatches).T

In [ ]:
from datetime import datetime
log_dir = 'logs'
os.makedirs(log_dir, exist_ok=True)
timestamp = datetime.utcnow().strftime('%Y-%m-%dT%H%M%SZ')
log_path = os.path.join(log_dir, f'{timestamp}.csv')
row = {'timestamp': timestamp, 'score': score}
row.update({f'{m}_value': d['value'] for m, d in mismatches.items()})
pd.DataFrame([row]).to_csv(log_path, index=False)
print(f'Logged to {log_path}')
